## Vector and Tracer Animations
This notebook loads and animates Salish Sea NEMO 3.4 Nowcast results. It requires having the XArray and Basemap packages installed.

In [1]:
import numpy                as np
import xarray               as xr
import datetime             as dtm
import dateutil.parser      as dparser
import matplotlib.pyplot    as plt
import matplotlib.animation as animation
import os
import collections

from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Rectangle

In [2]:
def load_NEMO(
    timerange, depth, w_grid, spacing,
    ERDDAP_url='https://salishsea.eos.ubc.ca/erddap/griddap'
):
    """Loads and processes Salish Sea NEMO 3.4 salinity and velocity fields
    on the specified timerange, depth, domain window, and vector spacing.
    """
    
    # Slice objects for xarray indexing
    tslice = slice(timerange[0], timerange[1])
    xslice = slice(w_grid[0], w_grid[1])
    yslice = slice(w_grid[2], w_grid[3])
    
    # Load Salish Sea NEMO results from ERDDAP server as xarray DataSets
    NEMO_grid = xr.open_dataset(os.path.join(ERDDAP_url, 'ubcSSnBathymetry2V1'))
    NEMO_trc  = xr.open_dataset(os.path.join(ERDDAP_url, 'ubcSSn3DTracerFields1hV1'))
    NEMO_u    = xr.open_dataset(os.path.join(ERDDAP_url, 'ubcSSn3DuVelocity1hV1'))
    NEMO_v    = xr.open_dataset(os.path.join(ERDDAP_url, 'ubcSSn3DvVelocity1hV1'))
    
    # Extract results as xarray DataArrays
    lon      = NEMO_grid.longitude.sel(gridX=xslice, gridY=yslice)
    lat      = NEMO_grid.latitude.sel( gridX=xslice, gridY=yslice)
    salinity = NEMO_trc.salinity.sel(  gridX=xslice, gridY=yslice, time=tslice).sel(
                                       depth=depth, method='nearest')
    u        = NEMO_u.uVelocity.sel(   gridX=xslice, gridY=yslice, time=tslice).sel(
                                       depth=depth, method='nearest')
    v        = NEMO_v.vVelocity.sel(   gridX=xslice, gridY=yslice, time=tslice).sel(
                                       depth=depth, method='nearest')
    
    # Unstagger currents
    u = np.add(u[..., :-1   ], u[..., 1:   ]) / 2
    v = np.add(v[..., :-1, :], v[..., 1:, :]) / 2
    v = v.reindex_like(u)
    
    # Subsample currents
    u = u[:, ::spacing, ::spacing]
    v = v[:, ::spacing, ::spacing]
        
    # Rotate currents
    theta_rad = 29 * np.pi / 180
    u_map = u * np.cos(theta_rad) - v * np.sin(theta_rad)
    v_map = u * np.sin(theta_rad) + v * np.cos(theta_rad)
    
    # Return as dict
    NEMO = {'lon': lon, 'lat': lat, 'salinity': salinity, 'u': u_map, 'v': v_map}
    
    return NEMO

In [3]:
def make_figure(ax, cmap, time_ind, w_map, spacing, NEMO):
    """Plots horizontal Salish Sea NEMO salinity and velocity fields
    on a basemap instance at the specified time index, map window, and
    vector spacing. (Vector spacing must be the SAME as that used in
    load_NEMO!)
    """
    
    # Find center coords
    lon_0 = (w_map[1] - w_map[0]) / 2 + w_map[0]
    lat_0 = (w_map[3] - w_map[2]) / 2 + w_map[2]
    
    # Plot map
    m = Basemap(projection='lcc', resolution='h', lon_0=lon_0, lat_0=lat_0,
                llcrnrlon=w_map[0], urcrnrlon=w_map[1],
                llcrnrlat=w_map[2], urcrnrlat=w_map[3])
    m.drawcoastlines(zorder=1)
    m.fillcontinents(color='burlywood', lake_color='white')
    m.drawmeridians(np.arange(w_map[0], w_map[1], 0.5), labels=[0, 0, 0, 1])
    m.drawparallels(np.arange(w_map[2], w_map[3], 0.5), labels=[1, 0, 0, 0])
    
    # Get map coordinates
    NEMO['x_map'], NEMO['y_map'] = m(NEMO['lon'].values, NEMO['lat'].values)
    
    # Plot salinity
    CONT_OBJ = m.contourf(NEMO['x_map'], NEMO['y_map'],
                np.ma.masked_values(NEMO['salinity'].sel(time=time_ind, method='nearest'), 0),
                range(32), cmap=cmap, zorder=0)
    cbar = m.colorbar(CONT_OBJ)
    cbar.set_label('Practical Salinity')
    
    # Plot currents
    QUIV_OBJ = m.quiver(
                NEMO['x_map'][1::spacing, 1::spacing], NEMO['y_map'][1::spacing, 1::spacing],
                np.ma.masked_values(NEMO['u'].sel(time=time_ind, method='nearest'), 0),
                np.ma.masked_values(NEMO['v'].sel(time=time_ind, method='nearest'), 0),
                scale=10)
    ax.add_patch(Rectangle((0.78, 0.9), 0.22, 0.1,
                facecolor='white', transform=ax.transAxes, zorder=4))
    Qkey = plt.quiverkey(QUIV_OBJ, 0.87, 0.93, 1, '1 m/s', coordinates='axes')
    Qkey.set_zorder(5)

    # Timestamp
    TEXT_OBJ = plt.text(0.02, 1.02, time_ind.strftime('%a %Y-%m-%d %H:%M:%S %Z'),
               transform=ax.transAxes, zorder=5)
    
    # Return as dict
    PLOT_OBJS = {'CONT_OBJ': CONT_OBJ, 'QUIV_OBJ': QUIV_OBJ, 'TEXT_OBJ': TEXT_OBJ}
    
    return NEMO, PLOT_OBJS

In [4]:
def update_figure(ax, cmap, time_ind, NEMO, PLOT_OBJS):
    """Updates horizontal Salish Sea NEMO salinity and velocity fields
    to the specified time index.
    """
    
    # Update NEMO salinity contours
    for C in PLOT_OBJS['CONT_OBJ'].collections: C.remove()
    PLOT_OBJS['CONT_OBJ'] = ax.contourf(
            NEMO['x_map'], NEMO['y_map'],
            np.ma.masked_values(NEMO['salinity'].sel(time=time_ind, method='nearest'), 0),
            range(32), cmap=cmap, zorder=0)
    
    # Update NEMO current vectors
    PLOT_OBJS['QUIV_OBJ'].set_UVC(
            np.ma.masked_values(NEMO['u'].sel(time=time_ind, method='nearest'), 0),
            np.ma.masked_values(NEMO['v'].sel(time=time_ind, method='nearest'), 0))
    
    # Update timestamp
    PLOT_OBJS['TEXT_OBJ'].set_text(time_ind.strftime('%a %Y-%m-%d %H:%M:%S %Z'))
    
    return PLOT_OBJS

In [5]:
def make_animation(
    timerange, depth=0, spacing=5, writepath='~/Desktop',
    w_grid=[100, 398, 230, 570], w_map=[-124, -122.7, 48.3, 49.7],
):
    """Animates Salish Sea NEMO 3.4 salinity and velocity fields over a
    Basemap instance on the specified timerange, depth, vector spacing,
    domain window, and map window. Saves to the the specified writepath.
    """
    
    # Parse dates
    starttime, endtime = map(dparser.parse, timerange)
    frames = (endtime - starttime).days * 24
    
    # Write file path
    writefile = 'NEMO_{}m_{}to{}.mp4'.format(
                depth, starttime.strftime('%Y%b%dT%H'), endtime.strftime('%Y%b%dT%H'))
    filepath = os.path.join(writepath, writefile)
    
    # Load NEMO
    NEMO = load_NEMO(timerange, depth, w_grid, spacing)
    
    # Make figure
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    cmap = plt.get_cmap('jet')
    cmap.set_bad('white')
    
    # Plot first timestep
    NEMO, PLOT_OBJS = make_figure(ax, cmap, starttime, w_map, spacing, NEMO)
    
    # Next frame definition
    def next_frame(t, PLOT_OBJS):
        time_ind = (starttime + dtm.timedelta(hours=t))
        PLOT_OBJS = update_figure(ax, cmap, time_ind, NEMO, PLOT_OBJS)
        return PLOT_OBJS
    
    # Animate
    mywriter = animation.FFMpegWriter(fps=12, bitrate=10000)
    ANI = animation.FuncAnimation(fig, next_frame, frames=frames, fargs=[PLOT_OBJS], blit=False)
    ANI.save(filepath, writer=mywriter)

In [6]:
timerange = ['2016 Jun 1 00:00', '2016 Jun 2 00:00']
writepath='/ocean/bmoorema/research/programming/swc/30Jun2016'
make_animation(timerange, writepath=writepath)